In [1]:
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
from numpy import load
from keras import losses

In [3]:
X_data=load('LSTM_input.npy')

In [13]:
pos_over=load('FO1_f5.npy')
pos_over2=load('FO1_l5.npy')

In [18]:
pos_over_total=np.concatenate((pos_over,pos_over2))
pos_over_total.shape
pos_over_LSTM=pos_over_total.reshape((100,20,512))

In [21]:
pos_over_pos=pos_over_LSTM[1:11,:,:]
print(pos_over_pos.shape)

(10, 20, 512)


In [22]:
over_sample=[]
for i in range(10):
    if i == 0:
        over_sample=pos_over_pos
    else:
        over_sample=np.concatenate((over_sample,pos_over_pos))
    print(over_sample.shape)

(10, 20, 512)
(20, 20, 512)
(30, 20, 512)
(40, 20, 512)
(50, 20, 512)
(60, 20, 512)
(70, 20, 512)
(80, 20, 512)
(90, 20, 512)
(100, 20, 512)


In [4]:
print(X_data.shape)

(500, 20, 512)


In [23]:
X_data_over_sample=np.concatenate((X_data,over_sample))
print(X_data_over_sample.shape)

(600, 20, 512)


In [25]:
def labeller(personID):
    if personID==0:
        Y_train_p=np.ones((10,1))
        Y_train_n=np.zeros((90,1))
        Y_data=np.concatenate((Y_train_p,Y_train_n))
    else:
        Y_data = np.zeros((100,1))
    return Y_data

In [6]:
count=0
for i in range(5):
    count=count+1
    Y_data=labeller(i)
    print(Y_data.shape)
    if count==1:
        Y_train_total=Y_data
    else:
        Y_train_total=np.concatenate((Y_train_total,Y_data))
    print(Y_train_total.shape)


(100, 1)
(100, 1)
(100, 1)
(200, 1)
(100, 1)
(300, 1)
(100, 1)
(400, 1)
(100, 1)
(500, 1)


In [28]:
Y_train_total2=np.concatenate((Y_train_total,np.ones((100,1))))
Y_train_total2.shape

(600, 1)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_data_over_sample,Y_train_total2, test_size=0.2)

X_train = []
X_test = []
y_train = []
y_test = []

w=0
f=0
for i in range(len(X_data)):
    if (i%9 == 0 or i%10 == 0):
        #put it in X_test, y_test
        if w==0:
            X_test=X_data[i]
            y_test=Y_train_total[i]
            w=w+1
        else:
            X_test=np.concatenate((X_test,X_data[i]))
            y_test=np.concatenate((y_test,Y_train_total[i]))
    else:
        if f==0:
            X_train=X_data[i]
            y_train=Y_train_total[i]
            f=f+1
        else:
            X_train=np.concatenate((X_train,X_data[i]))
            y_train=np.concatenate((y_train,Y_train_total[i]))
        #put it in X_train, y_train

In [31]:
model = Sequential()

input_shape=(X_train.shape[1],X_train.shape[2])

model.add(LSTM(units=11,return_sequences=True,input_shape=input_shape))
model.add(Dropout(0.2))

model.add(LSTM(units=16,return_sequences=True))
model.add(Dropout(0.2))


model.add(LSTM(units=16,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=16,input_shape=input_shape)) #16 units previously
model.add(Dropout(0.2))

#model.add(Dense(units=4))
model.add(Dense(units=1, activation='sigmoid'))

In [42]:
model.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['accuracy'])
model.fit(X_train,y_train,epochs=50,batch_size=75)


Epoch 1/50
480/480 [==============================] - 14s 29ms/step - loss: 0.1329 - acc: 0.9583
Epoch 2/50
480/480 [==============================] - 1s 3ms/step - loss: 0.1783 - acc: 0.9396
Epoch 3/50
480/480 [==============================] - 1s 3ms/step - loss: 0.1194 - acc: 0.9688
Epoch 4/50
480/480 [==============================] - 1s 3ms/step - loss: 0.1199 - acc: 0.9646A: 0s - loss: 0.1223 - acc: 0.
Epoch 5/50
480/480 [==============================] - 1s 3ms/step - loss: 0.1101 - acc: 0.9667
Epoch 6/50
480/480 [==============================] - 1s 3ms/step - loss: 0.1088 - acc: 0.9667
Epoch 7/50
480/480 [==============================] - 1s 3ms/step - loss: 0.1129 - acc: 0.9646
Epoch 8/50
480/480 [==============================] - 1s 3ms/step - loss: 0.0983 - acc: 0.9667
Epoch 9/50
480/480 [==============================] - 1s 3ms/step - loss: 0.1138 - acc: 0.9646
Epoch 10/50
480/480 [==============================] - 1s 3ms/step - loss: 0.1253 - acc: 0.9521
Epoch 11/50
480/

In [44]:
model.predict(X_test) 

array([[7.39902258e-04],
       [7.43746758e-04],
       [7.44044781e-04],
       [7.41064548e-04],
       [7.42286444e-04],
       [7.47144222e-04],
       [1.38691068e-03],
       [9.95861948e-01],
       [7.43508339e-04],
       [7.40498304e-04],
       [7.39991665e-04],
       [7.36743212e-04],
       [9.95861948e-01],
       [2.14764714e-01],
       [2.94793665e-01],
       [7.39544630e-04],
       [7.38620758e-04],
       [7.40915537e-04],
       [7.40110874e-04],
       [7.48395920e-04],
       [1.66117013e-01],
       [7.38412142e-04],
       [7.42197037e-04],
       [7.37667084e-04],
       [3.00736547e-01],
       [7.44909048e-04],
       [7.46607780e-04],
       [7.38650560e-04],
       [9.95861948e-01],
       [7.40110874e-04],
       [9.93950665e-01],
       [7.39306211e-04],
       [7.36951828e-04],
       [7.45087862e-04],
       [7.39157200e-04],
       [7.14072585e-03],
       [9.95861948e-01],
       [7.48425722e-04],
       [7.42107630e-04],
       [7.39663839e-04],


In [40]:
print(y_test) 

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [45]:
scores=model.evaluate(X_test,y_test,verbose=0)

In [46]:
print(scores)

[0.014267083443701267, 1.0]
